### ДЗ к уроку 4

In [73]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .master("local[2]")\
    .appName("Lesson_2")\
    .config("spark.executor.instances",2)\
    .config("spark.executor.memory",'2g')\
    .config("spark.executor.cores",1)\
    .getOrCreate()

In [74]:
df = spark.read.option('header','True').option('inferSchema', True).csv('raw_sales.csv',sep=',')

In [75]:
df.show(5)

+-------------------+--------+------+------------+--------+
|           datesold|postcode| price|propertyType|bedrooms|
+-------------------+--------+------+------------+--------+
|2007-02-07 00:00:00|    2607|525000|       house|       4|
|2007-02-27 00:00:00|    2906|290000|       house|       3|
|2007-03-07 00:00:00|    2905|328000|       house|       3|
|2007-03-09 00:00:00|    2905|380000|       house|       4|
|2007-03-21 00:00:00|    2906|310000|       house|       3|
+-------------------+--------+------+------------+--------+
only showing top 5 rows



In [76]:
df.count()

29580

<b>Задание 1</b><br>
Добавьте к таблице следующие поля: <br>
1. Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode)
2. Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)
3. Стоимость последнего проданного дома до текущего


In [77]:
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import lag

In [78]:
windowSpec  = Window()\
    .partitionBy("postcode")\
    .orderBy("datesold")\
    .rowsBetween(-10, 0)

df = df.withColumn('mean_10_before', F.avg(F.col('price')).over(windowSpec))

In [79]:
windowSpec  = Window()\
    .partitionBy("postcode")\
    .orderBy("datesold")\
    .rowsBetween(0, 10)

df = df.withColumn('mean_10_after', F.avg(F.col('price')).over(windowSpec))

In [80]:
windowSpec  = Window()\
    .partitionBy("postcode")\
    .orderBy("datesold")

df = df.withColumn('last_sold', lag('price', 1).over(windowSpec))

In [81]:
df.show(3)

+-------------------+--------+------+------------+--------+-----------------+-----------------+---------+
|           datesold|postcode| price|propertyType|bedrooms|   mean_10_before|    mean_10_after|last_sold|
+-------------------+--------+------+------------+--------+-----------------+-----------------+---------+
|2007-07-08 00:00:00|    2600|327000|       house|       1|         327000.0|673681.8181818182|     null|
|2007-08-16 00:00:00|    2600|790000|       house|       4|         558500.0|706681.8181818182|   327000|
|2007-12-05 00:00:00|    2600|825000|       house|       3|647333.3333333334|692590.9090909091|   790000|
+-------------------+--------+------+------------+--------+-----------------+-----------------+---------+
only showing top 3 rows



<b>Задание 2</b><br>
В итоге у вас таблица с колонками (или нечто похожее):<br>

price<br>
Среднегодовая цена<br>
Средняя стомость 10 проданных домов до текущего в том же районе (4digit postcode) (1 балл)<br>
Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode) (1 балл)<br>
Стоимость последнего проданного дома до текущего ((1 балл)<br>
и др.<br>
Посчитайте кол-во уникальных значений в каждой строчке (unique(row)) (ипользуйте udf). Попробуйте сделать то же самое используя pandas udf.

In [101]:
from pyspark.sql.functions import udf

In [82]:
df = df.withColumn('row_content', F.array(df.datesold, df.postcode, df.price, df.propertyType, df.bedrooms, df.last_sold, df.mean_10_after, df.mean_10_before))

In [90]:


def f(value):
    return len(set(value))

udf_f = F.udf(f, returnType=IntegerType())

df = df.withColumn('uniq_count', udf_f(df.row_content))

In [91]:
df.show()

Py4JJavaError: An error occurred while calling o948.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 104.0 failed 1 times, most recent failure: Lost task 0.0 in stage 104.0 (TID 72) (ASUSUX305C.mshome.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:188)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:108)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:121)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:162)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:130)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:863)
	at org.apache.spark.rdd.RDD.$anon

<font color='red'> К сожалению, тут опять вываливается моя любимая ошибка. Я попробовала сделать такую же функцию, как мы делали на уроке, но и она не сработала. Из чего я делаю вывод, что все-таки это не я где-то косячу, а это та самая ошибка.

In [103]:
@F.udf(returnType=StringType())
def f(value):
    return value.upper()

df.withColumn('test', f_udf(df.propertyType)).show()

Py4JJavaError: An error occurred while calling o1082.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 119.0 failed 1 times, most recent failure: Lost task 0.0 in stage 119.0 (TID 82) (ASUSUX305C.mshome.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:188)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:108)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:121)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:162)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:130)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:863)
	at org.apache.spark.rdd.RDD.$ano

<b>Задание 3</b><br>
SQL like case when или if elif else<br>

Создайте колонку, в которой в которой будет отображаться "+", "-" или "=", если "Средняя стомость 10 проданных домов до текущего в том же районе" больше, меньше или равно "Средняя стомость 10 проданных домов после текущего в том же районе (4digit postcode)", соотвественно.<br>

Если одно из полей Null, запишите в эту колонку "Нет данных"

In [110]:
from pyspark.sql.functions import when
df2 = df.withColumn("comparison", when(df.mean_10_before > df.mean_10_after,"+")
                                 .when(df.mean_10_before < df.mean_10_after,"-")
                                 .when(df.mean_10_before == df.mean_10_after,"=")
                                 .when(df.mean_10_before.isNull() ,"нет данных")
                                 .otherwise('нет данных'))

In [111]:
df2.show()

Py4JJavaError: An error occurred while calling o1125.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 125.0 failed 1 times, most recent failure: Lost task 0.0 in stage 125.0 (TID 86) (ASUSUX305C.mshome.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:188)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:108)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:121)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:162)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:81)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:130)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:863)
	at org.apache.spark.rdd.RDD.$ano